In [ ]:
import pandas as pd
import scanpy as sc
import seaborn as sns

In [ ]:
#from statannot import add_stat_annotation
#from statannotations.Annotator import Annotator

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import decoupler as dc

### Data loading

In [ ]:
adata = sc.read_h5ad(
    "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/13_january_2025/mapped_data/mapped_til_colon.h5ad"
)

In [ ]:
# Replace values in the 'origin' column
adata.obs["origin"] = adata.obs["origin"].replace({"colon": "normal", "til": "tumor"})


In [ ]:
adata.obs["cell_type"] = adata.obs["cell_type"].replace("nan", "CD8_NaiveLike")

# If 'nan' values are true NaN (e.g., from pandas)
adata.obs["cell_type"].fillna("CD8_NaiveLike", inplace=True)

In [ ]:
# Check if 'Ifng' is in the variable names
if 'ENSMUSG00000055170'in adata.var_names:
    # Safely extract the expression values for 'Ifng'
    ifng_expression = (
        adata[:, 'ENSMUSG00000055170'].X.toarray().flatten()
        if hasattr(adata[:, 'ENSMUSG00000055170'].X, "toarray")
        else adata[:, 'ENSMUSG00000055170'].X.flatten()
    )
    
    # Add Ifng expression status to adata.obs
    adata.obs['Ifng_expression'] = (ifng_expression > 6).astype(int)
    
    # Create a new column 'cell_type_2' based on 'Ifng_expression' and 'cell_type'
    adata.obs['cell_type_2'] = adata.obs.apply(
        lambda row: 'Ifng+' if row['Ifng_expression'] == 1 else row['cell_type'], axis=1
    )
else:
    print("Gene 'Ifng' not found in adata.var_names")

In [ ]:
# Create new column based on Ifng_expression values
adata.obs["Ifng_expression_str"] = adata.obs["Ifng_expression"].map({1: "pos", 0: "neg"})


In [ ]:
adata.obs['Ifng_expression_str'].value_counts()

In [ ]:
adata.obs['cell_type_2'].value_counts()

In [ ]:
adata.obs.condition.value_counts()

In [ ]:
adata.obs.cell_type.value_counts()

In [ ]:
adata_ifng = adata[adata.obs["cell_type_2"]=="Ifng+"]

In [ ]:
import scanpy as sc
import decoupler as dc

# Only needed for processing
import numpy as np
import pandas as pd

# Needed for some plotting
import matplotlib.pyplot as plt

In [ ]:
# Get filtered pseudo-bulk profile
pdata = dc.get_pseudobulk(
    adata_ifng,
    sample_col='sample_id',
    groups_col='cell_type',
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)
pdata

In [ ]:
pdata = pdata[pdata.obs["condition"].isin(["10mix","11mix"])]

In [ ]:
#pdata.var_names = pdata.var.gene_name

In [ ]:
pdata.obs

In [ ]:
# Import DESeq2
from pydeseq2.dds import DeseqDataSet, DefaultInference
from pydeseq2.ds import DeseqStats

In [ ]:
#Build DESeq2 object
inference = DefaultInference(n_cpus=8)
dds = DeseqDataSet(
    adata=pdata,
    design_factors=['condition','origin'],
    ref_level=['condition','10mix'],
    refit_cooks=True,
    inference=inference,
)

In [ ]:
# Compute LFCs
dds.deseq2()

In [ ]:
# Extract contrast between COVID-19 vs normal
stat_res = DeseqStats(
    dds,
    contrast=["condition", '11mix', '10mix'],
    inference=inference,
)

In [ ]:
# Compute Wald test
stat_res.summary()

In [ ]:
# Extract results
results_df = stat_res.results_df
results_df

In [ ]:
results_df = results_df.merge(pdata.var[['gene_name']], left_index=True, right_index=True, how="left")

In [ ]:
# Set 'gene_id' as the index
results_df.set_index("gene_name", inplace=True)


In [ ]:
dc.plot_volcano_df(
    results_df,
    x='log2FoldChange',
    y='padj',
    sign_thr=0.05,
    lFCs_thr=0.5,
    top=50,
    figsize=(4, 4)
)

In [ ]:
# Define significance thresholds
sign_thr = 0.05  # Adjusted p-value threshold
lFCs_thr = 0.5   # Log2 fold-change threshold

# Filter results_df for significant genes
significant_genes_df = results_df[
    (results_df["padj"] < sign_thr) & (results_df["log2FoldChange"].abs() > lFCs_thr)
]


In [ ]:
significant_genes_df

In [ ]:
list(significant_genes_df.index)

In [ ]:
import pandas as pd

# Ensure pdata is available and correctly loaded
# Extract sample annotations (metadata)
metadata_df = pdata.obs

# Extract read counts (gene expression matrix)
read_counts_df = pd.DataFrame(pdata.X.toarray() if hasattr(pdata.X, "toarray") else pdata.X,
                              index=pdata.obs_names,
                              columns=pdata.var_names)

# Save sample annotations to CSV
metadata_df.to_csv("sample_annotations.csv", index=True)
#
# Save read counts to TSV
#read_counts_df.to_csv("read_counts.tsv", sep="\t", index=True)




In [ ]:
read_counts_df = read_counts_df.T 
# Add gene_id column from pdata.var["gene_name"]
read_counts_df.insert(0, "gene_name", pdata.var["gene_name"].values)  # Insert as the first column

In [ ]:
read_counts_df.index.names = ['gene_id']

In [ ]:
read_counts_df

In [ ]:

# Save read counts to TSV
read_counts_df.to_csv("read_counts.tsv", sep="\t", index=True)  # Index (gene IDs) is included